In [0]:
%scala

val storageAccount = "rfigueroar" // nombre de la cuenta de almacenamiento
val container = "rfigueroar" // nombre del contenedor
val sasKey = "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2024-12-30T10:14:00Z&st=2024-12-08T02:14:00Z&spr=https&sig=KpRxuSD0BjUMMS417O6UhxW09WOUpzLBjTlLDghAZMc%3D" // llave de conexion

val conf = "fs.azure.sas." + container + "." + storageAccount + "." + "blob.core.windows.net"

// Crear acceso directo
dbutils.fs.mount(source="wasbs://" + container + "@" + storageAccount + ".blob.core.windows.net", mountPoint="/mnt/" + container, extraConfigs=Map(conf -> sasKey))

storageAccount: String = rfigueroar
container: String = rfigueroar
sasKey: String = sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2024-12-30T10:14:00Z&st=2024-12-08T02:14:00Z&spr=https&sig=KpRxuSD0BjUMMS417O6UhxW09WOUpzLBjTlLDghAZMc%3D
conf: String = fs.azure.sas.rfigueroar.rfigueroar.blob.core.windows.net
res0: Boolean = true

In [0]:
## INGESTAR ARCHIVO PRODUCTO.PARQUET
producto = (spark.read.parquet("/mnt/rfigueroar/proyectofinal/capaplata/Producto.parquet"))
producto.createOrReplaceTempView("s_producto")

In [0]:
## INGESTAR ARCHIVO CATEGORIA.PARQUET
categoria = (spark.read.parquet("/mnt/rfigueroar/proyectofinal/capaplata/Categoria.parquet"))
categoria.createOrReplaceTempView("s_categoria")

In [0]:
## INGESTAR ARCHIVO SUBCATEGORIA.PARQUET
subcategoria = (spark.read.parquet("/mnt/rfigueroar/proyectofinal/capaplata/SubCategoria.parquet"))
subcategoria.createOrReplaceTempView("s_subcategoria")

In [0]:
## INGESTAR ARCHIVO SUCURSAL.PARQUET
sucursal = (spark.read.parquet("/mnt/rfigueroar/proyectofinal/capaplata/Sucursal.parquet"))
sucursal.createOrReplaceTempView("s_sucursal")

In [0]:
## INGESTAR ARCHIVO VENTASINTERNET.PARQUET
ventasinternet = (spark.read.parquet("/mnt/rfigueroar/proyectofinal/capaplata/VentasInternet.parquet"))
ventasinternet.createOrReplaceTempView("s_ventasinternet")

In [0]:
%sql

-- Eliminar tabla en caso exista
DROP TABLE IF EXISTS Fact_Product;

-- Crear Tabla
CREATE TEMP VIEW Fact_Product
AS
SELECT P.Cod_Producto, P.Nombre AS Producto, S.Nombre AS Subcategoria, C.Nombre AS Categoria
FROM s_producto P
JOIN s_subcategoria S ON P.Cod_Subcategoria = S.Cod_Subcategoria
JOIN s_categoria C ON S.Cod_Categoria = C.Cod_Categoria

In [0]:
dfFactProducto = spark.table("Fact_Product")
dfFactProducto.write.mode("overwrite").parquet("/mnt/rfigueroar/proyectofinal/capaoro/FactProduct.parquet")

In [0]:
# Eliminar acceso directo
dbutils.fs.unmount("/mnt/rfigueroar")

/mnt/rfigueroar has been unmounted.


True